# Demonstration of the topic coherence pipeline in Gensim

## Introduction

We will be using the `u_mass` and `c_v` coherence for two different LDA models: a "good" and a "bad" LDA model. The good LDA model will be trained over 50 iterations and the bad one for 1 iteration. Hence in theory, the good LDA model will be able come up with better or more human-understandable topics. Therefore the coherence measure output for the good LDA model should be more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable.

In [1]:
from __future__ import print_function

import os
import logging
import json
import warnings

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import numpy as np

from gensim.models import CoherenceModel, LdaModel
from gensim.corpora import Dictionary

warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

/Users/home/Desenvolvimento/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL
/Users/home/Desenvolvimento/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


### Set up corpus

As stated in table 2 from [this](http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf) paper, this corpus essentially has two classes of documents. First five are about human-computer interaction and the other four are about graphs. We will be setting up two LDA models. One with 50 iterations of training and the other with just 1. Hence the one with 50 iterations ("better" model) should be able to capture this underlying pattern of the corpus better than the "bad" LDA model. Therefore, in theory, our topic coherence for the good LDA model should be greater than the one for the bad LDA model.

In [2]:
texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

In [3]:
dictionary = Dictionary(texts)
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)
[[(0, 1), (1, 1), (2, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(1, 1), (4, 1), (5, 1), (8, 1)], [(0, 1), (5, 2), (8, 1)], [(4, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(3, 1), (10, 1), (11, 1)]]


### Set up two topic models

We'll be setting up two different LDA Topic models. A good one and bad one. To build a "good" topic model, we'll simply train it using more iterations than the bad one. Therefore the `u_mass` coherence should in theory be better for the good model than the bad one since it would be producing more "human-interpretable" topics.

In [4]:
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=2)
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=1, num_topics=2)

### Using U_Mass Coherence

In [5]:
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
badcm = CoherenceModel(model=badLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

### View the pipeline parameters for one coherence model

Following are the pipeline parameters for `u_mass` coherence. By pipeline parameters, we mean the functions being used to calculate segmentation, probability estimation, confirmation measure and aggregation as shown in figure 1 in [this](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) paper.

In [6]:
print(goodcm)

Coherence_Measure(seg=<function s_one_pre at 0x1163abe18>, prob=<function p_boolean_document at 0x116372d08>, conf=<function log_conditional_probability at 0x116407c80>, aggr=<function arithmetic_mean at 0x116408488>)


### Interpreting the topics

As we will see below using LDA visualization, the better model comes up with two topics composed of the following words:
1. goodLdaModel:
    - __Topic 1__: More weightage assigned to words such as "system", "user", "eps", "interface" etc which captures the first set of documents.
    - __Topic 2__: More weightage assigned to words such as "graph", "trees", "survey" which captures the topic in the second set of documents.
2. badLdaModel:
    - __Topic 1__: More weightage assigned to words such as "system", "user", "trees", "graph" which doesn't make the topic clear enough.
    - __Topic 2__: More weightage assigned to words such as "system", "trees", "graph", "user" which is similar to the first topic. Hence both topics are not human-interpretable.

Therefore, the topic coherence for the goodLdaModel should be greater for this than the badLdaModel since the topics it comes up with are more human-interpretable. We will see this using `u_mass` and `c_v` topic coherence measures.

### Visualize topic models

In [13]:
pyLDAvis.gensim.prepare(goodLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
0      65.034771        1       1  0.024834  0.0
1      34.965229        1       2 -0.024834  0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
10    Default  2.000000      graph  2.000000  12.0000  12.0000
11    Default  1.000000     minors  1.000000  11.0000  11.0000
9     Default  2.000000      trees  2.000000  10.0000  10.0000
3     Default  2.000000     survey  2.000000   9.0000   9.0000
1     Default  2.000000  interface  2.000000   8.0000   8.0000
6     Default  2.000000   response  2.000000   7.0000   7.0000
0     Default  2.000000      human  2.000000   6.0000   6.0000
7     Default  2.000000       time  2.000000   5.0000   5.0000
5     Default  3.000000     system  3.000000   4.0000   4.0000
2     Default  2.000000   computer  2.000000   3.0000   3.0000
4     Default  2.000000       user  2.000000   2.0000   2.0000
8     Default  2.000000        eps  2.000000   1.0000   1.0000
5      Topic1  3.185145     system  3.861379   0.2377  -1.7785
8      Topic1  1.854703        eps  2.296138   0.2167  -2.3193
4      Topic1  2.170511       user  2.950544   0.1232  -2.1621
2      Topic1  1.530448   computer  2.177155   0.0778  -2.5115
7      Topic1  1.516915       time  2.172189   0.0712  -2.5204
0      Topic1  1.417995      human  2.135891   0.0206  -2.5878
6      Topic1  1.407341   response  2.131982   0.0149  -2.5953
1      Topic1  1.350600  interface  2.111161  -0.0164  -2.6365
9      Topic1  1.685608      trees  2.772612  -0.0674  -2.4149
3      Topic1  1.221698     survey  2.063861  -0.0941  -2.7368
10     Topic1  0.951676      graph  2.503302  -0.5369  -2.9866
11     Topic1  0.567445     minors  1.823788  -0.7373  -3.5037
11     Topic2  1.256343     minors  1.823788   0.6781  -2.0883
10     Topic2  1.551626      graph  2.503302   0.5725  -1.8772
3      Topic2  0.842163     survey  2.063861   0.1545  -2.4883
9      Topic2  1.087005      trees  2.772612   0.1145  -2.2331
1      Topic2  0.760561  interface  2.111161   0.0299  -2.5902
6      Topic2  0.724641   response  2.131982  -0.0283  -2.6386
0      Topic2  0.717896      human  2.135891  -0.0395  -2.6479
7      Topic2  0.655274       time  2.172189  -0.1476  -2.7392
2      Topic2  0.646707   computer  2.177155  -0.1631  -2.7523
4      Topic2  0.780033       user  2.950544  -0.2796  -2.5649
8      Topic2  0.441435        eps  2.296138  -0.5981  -3.1342
5      Topic2  0.676234     system  3.861379  -0.6914  -2.7077, token_table=      Topic      Freq       Term
term                            
2         1  0.918630   computer
2         2  0.459315   computer
8         1  0.871028        eps
10        1  0.399472      graph
10        2  0.798945      graph
0         1  0.468189      human
0         2  0.468189      human
1         1  0.473673  interface
1         2  0.473673  interface
11        1  0.548309     minors
11        2  0.548309     minors
6         1  0.469047   response
6         2  0.469047   response
3         1  0.484529     survey
3         2  0.484529     survey
5         1  0.776925     system
5         2  0.258975     system
7         1  0.920730       time
7         2  0.460365       time
9         1  0.721341      trees
9         2  0.360671      trees
4         1  0.677841       user
4         2  0.338921       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [14]:
pyLDAvis.gensim.prepare(badLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
1      50.105875        1       1  0.009925  0.0
0      49.894125        1       2 -0.009925  0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
1     Default  2.000000  interface  2.000000  12.0000  12.0000
10    Default  2.000000      graph  2.000000  11.0000  11.0000
6     Default  2.000000   response  2.000000  10.0000  10.0000
8     Default  2.000000        eps  2.000000   9.0000   9.0000
5     Default  3.000000     system  3.000000   8.0000   8.0000
2     Default  2.000000   computer  2.000000   7.0000   7.0000
11    Default  2.000000     minors  2.000000   6.0000   6.0000
9     Default  2.000000      trees  2.000000   5.0000   5.0000
3     Default  2.000000     survey  2.000000   4.0000   4.0000
7     Default  2.000000       time  2.000000   3.0000   3.0000
0     Default  2.000000      human  2.000000   2.0000   2.0000
4     Default  2.000000       user  2.000000   1.0000   1.0000
1      Topic1  1.547704  interface  2.162430   0.3566  -2.2395
8      Topic1  1.291081        eps  2.140989   0.1852  -2.4208
2      Topic1  1.248261   computer  2.137411   0.1532  -2.4545
5      Topic1  2.044810     system  3.559376   0.1368  -1.9610
0      Topic1  1.145949      human  2.128863   0.0717  -2.5400
4      Topic1  1.434564       user  2.830683   0.0114  -2.3154
7      Topic1  0.968368       time  2.114026  -0.0897  -2.7084
9      Topic1  1.244004      trees  2.814762  -0.1255  -2.4579
3      Topic1  0.912880     survey  2.109390  -0.1465  -2.7674
11     Topic1  0.885521     minors  2.107104  -0.1759  -2.7978
6      Topic1  0.821070   response  2.101719  -0.2489  -2.8734
10     Topic1  0.986493      graph  2.793247  -0.3498  -2.6899
10     Topic2  1.806754      graph  2.793247   0.2596  -2.0805
6      Topic2  1.280650   response  2.101719   0.1999  -2.4247
11     Topic2  1.221583     minors  2.107104   0.1501  -2.4719
3      Topic2  1.196510     survey  2.109390   0.1283  -2.4926
9      Topic2  1.570758      trees  2.814762   0.1119  -2.2205
7      Topic2  1.145658       time  2.114026   0.0827  -2.5360
4      Topic2  1.396119       user  2.830683  -0.0116  -2.3383
0      Topic2  0.982914      human  2.128863  -0.0776  -2.6893
5      Topic2  1.514565     system  3.559376  -0.1592  -2.2569
2      Topic2  0.889150   computer  2.137411  -0.1818  -2.7895
8      Topic2  0.849908        eps  2.140989  -0.2286  -2.8347
1      Topic2  0.614726  interface  2.162430  -0.5625  -3.1586, token_table=      Topic      Freq       Term
term                            
2         1  0.467856   computer
2         2  0.467856   computer
8         1  0.467074        eps
8         2  0.467074        eps
10        1  0.358006      graph
10        2  0.716013      graph
0         1  0.469734      human
0         2  0.469734      human
1         1  0.924886  interface
1         2  0.462443  interface
11        1  0.474585     minors
11        2  0.474585     minors
6         1  0.475801   response
6         2  0.475801   response
3         1  0.474071     survey
3         2  0.474071     survey
5         1  0.561896     system
5         2  0.561896     system
7         1  0.473031       time
7         2  0.473031       time
9         1  0.355270      trees
9         2  0.710540      trees
4         1  0.353272       user
4         2  0.353272       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [10]:
print(goodcm.get_coherence())
print(badcm.get_coherence())

-14.6676987064
-14.7076308762


### Using C_V coherence

In [11]:
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
badcm = CoherenceModel(model=badLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')

### Pipeline parameters for C_V coherence

In [12]:
print(goodcm)

Coherence_Measure(seg=<function s_one_set at 0x10d5b5f28>, prob=<function p_boolean_sliding_window at 0x10d5a8e18>, conf=<function cosine_similarity at 0x10d60eea0>, aggr=<function arithmetic_mean at 0x10d611488>)


### Print coherence values

In [13]:
print(goodcm.get_coherence())
print(badcm.get_coherence())

0.383841355374
0.383841355374


### Conclusion

Hence as we can see, the `u_mass` and `c_v` coherence for the good LDA model is much more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable. The badLdaModel however fails to decipher between these two topics and comes up with topics which are not clear to a human. The `u_mass` and `c_v` topic coherences capture this wonderfully by giving the interpretability of these topics a number as we can see above. Hence this coherence measure can be used to compare difference topic models based on their human-interpretability.